In [ ]:
import os

# 1. Create a directory for the data
if not os.path.exists('data'):
    os.makedirs('data')

# 2. Download the zip file (using a reliable mirror for the NASA dataset)
!#wget https://data.nasa.gov/docs/legacy/CMAPSSData.zip -O data/CMAPSSData.zip

# 3. Unzip it
#!unzip -o data/CMAPSSData.zip -d data/

#print("Data downloaded and extracted!")

In [ ]:
import pandas as pd

In [ ]:
# The dataset has 26 columns
# 1. Unit Number (Which engine is it?)
# 2. Time Cycles (How long has it been running?)
# 3-5. Operational Settings (Altitude, Speed, etc.)
# 6-26. Sensor Readings (s1 to s21)

index_names = ['unit_nr', 'time_cycles']
setting_names = ['setting_1', 'setting_2', 'setting_3']
sensor_names = ['s_{}'.format(i) for i in range(1, 22)] 
col_names = index_names + setting_names + sensor_names

print(col_names)

In [ ]:
def load_fd(fd_tag):
    train_path = f"data/train_{fd_tag}.txt"
    test_path  = f"data/test_{fd_tag}.txt"
    rul_path   = f"data/RUL_{fd_tag}.txt"

    raw_train_df = pd.read_csv(train_path, sep=r'\s+', header=None, names=col_names)
    raw_test_df  = pd.read_csv(test_path,  sep=r'\s+', header=None, names=col_names)
    raw_rul_labels_df = pd.read_csv(rul_path, header=None, names=['RUL_truth'])

    # train labels: compute RUL from run-to-failure
    max_cycle = raw_train_df.groupby('unit_nr')['time_cycles'].max().rename('max_cycle')
    raw_train_df = raw_train_df.merge(max_cycle, left_on='unit_nr', right_index=True)
    raw_train_df['RUL'] = raw_train_df['max_cycle'] - raw_train_df['time_cycles']

    # test labels: provided separately
    return raw_train_df, raw_test_df, raw_rul_labels_df


In [ ]:
next_unit = 1
train_dfs = []
test_dfs = []
test_rul_labels = []

data_tags = ["FD001","FD002","FD003","FD004"]

for fd_tag in data_tags:
    train_df_chunk, test_df_chunk, rul_labels_chunk = load_fd(fd_tag)
    train_df_chunk['fd'] = fd_tag
    test_df_chunk['fd'] = fd_tag
    test_df_chunk = test_df_chunk.assign(
        unit_nr_orig=test_df_chunk['unit_nr'],
        unit_nr=test_df_chunk['unit_nr'] + next_unit - 1
    )
    test_dfs.append(test_df_chunk)
    test_rul_labels.append(rul_labels_chunk)

    # make a mapping for this FD's units
    uniq_units = sorted(train_df_chunk['unit_nr'].unique())
    mapping = {u: next_unit + i for i, u in enumerate(uniq_units)}
    next_unit += len(uniq_units)

    train_df_chunk = train_df_chunk.assign(
        unit_nr_orig=train_df_chunk['unit_nr'],
        unit_nr=train_df_chunk['unit_nr'].map(mapping),
        fd=fd_tag
    )
    train_dfs.append(train_df_chunk)
    
    
data_df = pd.concat(train_dfs, ignore_index=True)
train_df = pd.concat(train_dfs, ignore_index=True)
test_df = pd.concat(test_dfs, ignore_index=True)
rul_labels_df = pd.concat(test_rul_labels, ignore_index=True)

In [ ]:
train_df.info()


In [ ]:
import pandas as pd

train_df.head(10)


In [ ]:
test_df.info()


In [ ]:
rul_labels_df.info()

In [ ]:
from sklearn.preprocessing import MinMaxScaler


not_scaled_cols = ['unit_nr', 'RUL', 'max_cycle', 'time_cycles']

col_set = set(col_names)
columns_to_scale = [col for col in col_names if col not in not_scaled_cols]

print("Columns to scale:", columns_to_scale)

scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(train_df[columns_to_scale])
scaled_data_df = pd.DataFrame(scaled, columns=columns_to_scale, index=train_df.index)

scaled_data_df.insert(0, 'unit_nr', train_df['unit_nr'])
scaled_data_df.insert(1, 'time_cycles', train_df['time_cycles'])
scaled_data_df.insert(len(scaled_data_df.columns), 'RUL', train_df['RUL'])
scaled_data_df.insert(len(scaled_data_df.columns), 'max_cycle', train_df['max_cycle'])
scaled_data_df.head()


In [ ]:
engine_ids = scaled_data_df['unit_nr'].unique()
engine_ids

In [ ]:
train_ids = engine_ids[:650]
test_ids = engine_ids[650:]
print("len train ids:", len(train_ids))
print("len test ids:", len(test_ids))
print("len engine ids:", len(engine_ids))

In [ ]:
train_df = scaled_data_df[scaled_data_df['unit_nr'].isin(train_ids)]
test_df = scaled_data_df[scaled_data_df['unit_nr'].isin(test_ids)]
print("Train df shape:", train_df.shape)
print("Test df shape:", test_df.shape)

train_df.head()

In [ ]:
test_df.head()

In [ ]:
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view

def create_sequences_vectorized(X, y, unit_ids, seq_length=50):
    # 1. Create windows (Batch, Seq, Features)
    X_windows = sliding_window_view(X, window_shape=seq_length, axis=0)
    X_windows = X_windows.transpose(0, 2, 1) # (Batch, Seq, Features)
    
    # 2. Align Targets (End of window)
    y_aligned = y[seq_length-1:]
    
    # 3. Create Mask (Ensure window doesn't cross units)
    unit_ids_start = unit_ids[:-seq_length+1]
    unit_ids_end   = unit_ids[seq_length-1:]
    valid_mask = (unit_ids_start == unit_ids_end)
    
    return X_windows[valid_mask], y_aligned[valid_mask]

# --- Usage ---
sensor_cols = [c for c in train_df.columns if c.startswith('s_')]




# 1. Prepare arrays (Sorted)
train_df = train_df.sort_values(['unit_nr', 'time_cycles'])
test_df = test_df.sort_values(['unit_nr', 'time_cycles'])


# Apply rolling mean with window 9 (common for FD001)
#print("Smoothing sensor data...")
for col in sensor_cols:
    train_df[col] = train_df.groupby('unit_nr')[col].transform(
    lambda x: x.rolling(window=9, min_periods=1).mean())
    test_df[col] = test_df.groupby('unit_nr')[col].transform(
    lambda x: x.rolling(window=9, min_periods=1).mean())


# 2. CRITICAL: Drop Target (RUL) and max_cycle from Inputs
# We only want the 24 sensor/setting columns + time_cycle
features_to_drop = ['unit_nr', 'time_cycles', 'RUL', 'max_cycle', "s_1", "s_5", "s_10", "s_16", "s_18", "s_19"]

X_train_arr = train_df.drop(columns=features_to_drop).values
y_train_arr = train_df['RUL'].values 
train_units = train_df['unit_nr'].values

X_test_arr = test_df.drop(columns=features_to_drop).values
y_test_arr = test_df['RUL'].values
test_units = test_df['unit_nr'].values

# 3. Create Sequences
X_train_seq, y_train_seq = create_sequences_vectorized(X_train_arr, y_train_arr, train_units, 50)
X_test_seq, y_test_seq = create_sequences_vectorized(X_test_arr, y_test_arr, test_units, 50)

print(f"Train Input Shape: {X_train_seq.shape}") 
# Expected shape: (N, 50, 24) -> 24 features

import torch


X_train_tensor = torch.tensor(X_train_seq, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_seq, dtype=torch.float32)

X_test_tensor = torch.tensor(X_test_seq, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_seq, dtype=torch.float32)   

print(f"X_train_tensor.shape: {X_train_tensor.shape}")
print(f"y_train_tensor.shape: {y_train_tensor.shape}")

print(f"X_test_tensor.shape: {X_test_tensor.shape}")
print(f"y_test_tensor.shape: {y_test_tensor.shape}")
#print(f"X_test_tensor[0]: {X_test_tensor[0]}")
#print(f"y_test_tensor[0]: {y_test_tensor[0]}")

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
from sklearn.metrics import mean_squared_error
import copy
from model import EngineRULPredictor

# 2. Setup (Reduced hidden size slightly to 128 to prevent overfitting)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


model = EngineRULPredictor(input_size=X_train_tensor.shape[2], hidden_size=512, num_layers=2, dropout=0.2)
model = model.to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=5
)

# 3. Training with "Save Best" logic
EPOCHS = 10  # Lower epochs, let early stopping do the work
best_test_rmse = float('inf')
best_model_wts = copy.deepcopy(model.state_dict())
max_rul = train_df['RUL'].max()

y_train_tensor = torch.tensor(y_train_seq / max_rul, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_seq / max_rul, dtype=torch.float32)

# Move validation data to GPU once
X_test_gpu = X_test_tensor.to(device)
y_test_real = y_test_seq # Keep real values for RMSE calculation


train_loader = DataLoader(
    TensorDataset(X_train_tensor, y_train_tensor),
    batch_size=256,
    shuffle=True,
    drop_last=True
)


print("Starting Training with Validation...")

for epoch in range(EPOCHS):
    model.train() # Set to training mode (enables Dropout)
    epoch_loss = 0
    
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        out = model(X_batch)
        loss = criterion(out.squeeze(), y_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        
    # --- Validation Step ---
    model.eval() # Set to eval mode (disables Dropout)
    with torch.no_grad():
        # Get predictions
        preds_scaled = model(X_test_gpu).cpu().numpy().flatten()
        # Unscale
        preds_real = preds_scaled * max_rul
        # Calculate true RMSE
        mse = mean_squared_error(y_test_real, preds_real)
        current_rmse = np.sqrt(mse)
        scheduler.step(current_rmse)
    
    # Save model if it's the best so far
    if current_rmse < best_test_rmse:
        best_test_rmse = current_rmse
        best_model_wts = copy.deepcopy(model.state_dict())
        print(f"Epoch {epoch+1}: New Best RMSE: {current_rmse:.2f}")
    
    if (epoch + 1) % 10 == 0:
        avg_train_loss = epoch_loss / len(train_loader)
        print(f"Epoch {epoch+1}: Train Loss {avg_train_loss:.6f} | Test RMSE {current_rmse:.2f}")

# 4. Load the best weights back
print(f"Training complete. Best RMSE: {best_test_rmse:.2f}")
model.load_state_dict(best_model_wts)

In [ ]:
from model import EngineRULPredictor

X_test_tensor = X_test_tensor.to("cpu")
y_test_tensor = y_test_tensor.to("cpu")


input_size = 18 # Sensors (21) + Settings (3)
model = EngineRULPredictor(input_size=input_size, hidden_size=512, num_layers=2, dropout=0.2)

model.load_state_dict(torch.load('models/lstm_model.pth', map_location=torch.device('cpu')))
model.eval()

model.to("cpu")

#X_test_tensor = torch.tensor(X_test_seq, dtype=torch.float32).to(device)
# y_test_seq is the original TRUE RUL (not scaled)

model.eval()
with torch.no_grad():
    # Predict (Output is 0-1)
    preds_scaled = model(X_test_tensor)
    # Un-scale (Output becomes 0-300)
    preds_real = preds_scaled.cpu().numpy().flatten() * max_rul

# Calculate RMSE on real values
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test_seq, preds_real)
print(f"Test RMSE: {np.sqrt(mse):.2f}")

from matplotlib import pyplot as plt

# Plot
plt.figure(figsize=(10,6))
plt.plot(y_test_seq[:400], label='True RUL')
plt.plot(preds_real[:400], label='Predicted RUL')
plt.legend()
plt.show()

In [ ]:
from model import EngineRULPredictor

X_test_tensor = X_test_tensor.to("cpu")
y_test_tensor = y_test_tensor.to("cpu")

print(f"X_test_tensor.shape: {X_test_tensor.shape}")
print(f"y_test_tensor.shape: {y_test_tensor.shape}")

print(f"y_test_tensor[0]: {y_test_tensor[0] * max_rul}")

print(f"y_test_seq[0]: {y_test_seq[0]}")

In [ ]:
input_size = 18 # Sensors (21) + Settings (3)
model = EngineRULPredictor(input_size=input_size, hidden_size=512, num_layers=2, dropout=0.2)

model.load_state_dict(torch.load('models/lstm_model.pth', map_location=torch.device('cpu')))
model.eval()

model.to("cpu")

#X_test_tensor = torch.tensor(X_test_seq, dtype=torch.float32).to(device)
# y_test_seq is the original TRUE RUL (not scaled)

model.eval()
with torch.no_grad():
    # Predict (Output is 0-1)
    preds_scaled = model(X_test_tensor)
    # Un-scale (Output becomes 0-300)
    preds_real = preds_scaled.cpu().numpy().flatten() * max_rul

# Calculate RMSE on real values
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test_seq, preds_real)
print(f"Test RMSE: {np.sqrt(mse):.2f}")

from matplotlib import pyplot as plt

# Plot
plt.figure(figsize=(10,6))
plt.plot(y_test_seq[:400], label='True RUL')
plt.plot(preds_real[:400], label='Predicted RUL')
plt.legend()
plt.show()

In [ ]:
import os
import joblib
import torch

# 1. Create a folder to keep your project clean
if not os.path.exists('models'):
    os.makedirs('models')

# 2. Save the Scaler using joblib
joblib.dump(scaler, 'models/scaler.pkl')
print("Scaler saved to models/scaler.pkl")

# 3. Save the Model using torch
torch.save(best_model_wts, 'models/lstm_model.pth')
print("Model weights saved to models/lstm_model.pth")

In [ ]:
from pydantic import BaseModel


class EngineData(BaseModel):
    unit_nr: int
    time_cycles: int
    setting_1: float
    setting_2: float
    setting_3: float
    s_1: float
    s_2: float
    s_3: float
    s_4: float
    s_5: float
    s_6: float
    s_7: float
    s_8: float
    s_9: float
    s_10: float
    s_11: float
    s_12: float
    s_13: float
    s_14: float
    s_15: float
    s_16: float
    s_17: float
    s_18: float
    s_19: float
    s_20: float
    s_21: float

In [ ]:
# pick whichever frame you want to validate (raw or scaled)
df_for_schema = data_df[col_names]          # or scaled_data_df[col_names]

records = df_for_schema.to_dict(orient="records")
engine_rows = [EngineData.model_validate(rec) for rec in records]

# single row example
first_engine = EngineData.model_validate(df_for_schema.iloc[0].to_dict())


In [ ]:
df_for_schema.head()

In [ ]:
print(records[0])
print(first_engine)

In [ ]:
print(f"len engine_rows: {len(engine_rows)}")

print(f"engine_rows[0]: {engine_rows[0]}")

In [ ]:
from typing import List
from pydantic import BaseModel, Field

class InferencePayload(BaseModel):
    engine_data_sequence: List[EngineData] = Field(min_length=1, max_length=50)

    def to_dataframe(self) -> pd.DataFrame:
        data_dicts = [edata.model_dump() for edata in self.engine_data_sequence]
        return pd.DataFrame(data_dicts)


In [ ]:
from model import EngineRULPredictor
import torch
import pandas as pd
import joblib

input_size = 18 # Sensors (21) + Settings (3)
model = EngineRULPredictor(input_size=input_size, hidden_size=512, num_layers=2, dropout=0.2)

model.load_state_dict(torch.load('models/lstm_model.pth', map_location=torch.device('cpu')))
model.eval()

scaler = joblib.load('models/scaler.pkl') # <--- 2. Load the scaler



index_names = ['unit_nr', 'time_cycles']
setting_names = ['setting_1', 'setting_2', 'setting_3']
sensor_names = ['s_{}'.format(i) for i in range(1, 22)] 
col_names = index_names + setting_names + sensor_names
df = pd.read_csv("data/train_FD001.txt", sep="\s+", header=None, names=col_names)
df = df.head(50)  # Use only the first 50 rows for prediction

# Convert payload to DataFrame
#df = payload.to_dataframe()

df = df.drop(columns=["unit_nr", "time_cycles"])  # Drop non-feature columns for scaling


df.head()


In [ ]:
#features_to_drop = ['unit_nr', 'time_cycles', "s_1", "s_5", "s_10", "s_16", "s_18", "s_19"]
#df = df.drop(columns=features_to_drop)
scaled = scaler.transform(df)

df = pd.DataFrame(scaled, columns=df.columns, index=df.index)

df.head()
features_to_drop = ["s_1", "s_5", "s_10", "s_16", "s_18", "s_19"]
df = df.drop(columns=features_to_drop)



In [ ]:
input_tensor = torch.tensor(df.to_numpy(), dtype=torch.float32)
input_tensor = input_tensor.unsqueeze(0)  # Add batch dimension

print(f"input_tensor.shape: {input_tensor.shape}")

# Perform prediction
with torch.no_grad():
    prediction = model(input_tensor)
    preds_real = prediction.cpu().numpy().flatten() * max_rul

    print(f"Predicted RUL (real scale): {preds_real}")

from matplotlib import pyplot as plt

# Plot
plt.figure(figsize=(10,6))
#plt.plot(y_test_seq[:400], label='True RUL')
plt.plot(preds_real, label='Predicted RUL')
plt.legend()
plt.show()


# Assuming the model outputs a single RUL value
rul_prediction = prediction.item()

In [ ]:
rul_prediction

In [ ]:
model.to("cpu")



scaler = joblib.load("models/scaler.pkl")
max_rul = 542  # use the same value you trained with

df = pd.read_csv("data/train_FD001.txt", sep=r"\s+", header=None, names=col_names).head(50)
df_no_id = df.drop(columns=["unit_nr", "time_cycles"])
df_scaled = pd.DataFrame(scaler.transform(df_no_id), columns=df_no_id.columns, index=df.index)
df_model = df_scaled.drop(columns=["s_1","s_5","s_10","s_16","s_18","s_19"])

input_tensor = torch.tensor(df_model.to_numpy(), dtype=torch.float32).unsqueeze(0)
with torch.no_grad():
    rul_scaled = model(input_tensor).item()
rul_cycles = rul_scaled * max_rul

print(f"Predicted RUL (cycles): {rul_cycles:.2f}")


In [ ]:
from model import EngineRULPredictor

X_test_tensor = X_test_tensor.to("cpu")
y_test_tensor = y_test_tensor.to("cpu")

print(f"X_test_tensor.shape: {X_test_tensor.shape}")

input_size = 18 # Sensors (21) + Settings (3)
model = EngineRULPredictor(input_size=input_size, hidden_size=512, num_layers=2, dropout=0.2)

model.load_state_dict(torch.load('models/lstm_model.pth', map_location=torch.device('cpu')))
model.eval()

model.to("cpu")

#X_test_tensor = torch.tensor(X_test_seq, dtype=torch.float32).to(device)
# y_test_seq is the original TRUE RUL (not scaled)

model.eval()
with torch.no_grad():
    # Predict (Output is 0-1)
    preds_scaled = model(X_test_tensor)
    # Un-scale (Output becomes 0-300)
    preds_real = preds_scaled.cpu().numpy().flatten() * max_rul

# Calculate RMSE on real values
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test_seq, preds_real)
print(f"Test RMSE: {np.sqrt(mse):.2f}")

from matplotlib import pyplot as plt

# Plot
plt.figure(figsize=(10,6))
plt.plot(y_test_seq[:400], label='True RUL')
plt.plot(preds_real[:400], label='Predicted RUL')
plt.legend()
plt.show()

In [10]:
import pandas as pd    
unit_nr = 2

rul_df = pd.read_csv("data/RUL_FD001.txt", sep="\s+", header=None, names=["rul"])
rul_df.index = range(1, len(rul_df) + 1)
rul_df.index.name = "unit_nr"
rul_df = rul_df[rul_df.index == unit_nr]

rul_df.head()
#rul_series = rul_df["rul"]
#rul_series.head() 


<>:4: SyntaxWarning: invalid escape sequence '\s'
<>:4: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_2591124/3633571523.py:4: SyntaxWarning: invalid escape sequence '\s'
  rul_df = pd.read_csv("data/RUL_FD001.txt", sep="\s+", header=None, names=["rul"])


,rul
unit_nr,
2,98


In [13]:
rul_df.values.item()

98